In [60]:
from selenium import webdriver
import bs4
import pandas as pd
from time import sleep


title_names = [
    '공모전명','사 업 명','축 제 명','사 업 명','공 모 명'
]

date_names = [
    '공모기간', '일       시','일 시','축제 기간','응모기간','모집기간'
    ,'사업기간','접수기간','연수기간','공고기간','제공일자','일시'
]

place_names = [
    '장       소','장 소','축제 장소','장소',
]

content_names = [
    '내용','사업개요','사업내용','응모주제','사업목적','연수내용'
]

target_person_name = [
    '지원 대상','신청자격','지원대상','참여대상','참석 대상','참가 자격','모집대상',
    '자격요건',' 신청자격','공모대상'
]

body = []



In [61]:
title = []
date = []
place = []
content = []
target = []

In [10]:
#게시글의 제목만 파싱하는 코드 입니다.
#2019.11.14기준 JDC홈페이지의 웹 디자인이 변경됨에 따라 재작성 해야합니다.
sources = []

driver = webdriver.Chrome('C:\웹 드라이버\chromedriver.exe')
driver.implicitly_wait(1)
driver.get('https://www.jdcenter.com/prcenter/issue/notice.cs')

sleep(3)

for i in range(2,6):
    next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(i))
    next_page.click() 
    sleep(2)
    
html = driver.page_source
soup = bs4.BeautifulSoup(html, 'html.parser')

trs = soup.find_all('tr')
    
# <tr>태그 내부에 존재하는 <td>태그를 탐색합니다.
for tr in trs:
    tds = tr.find_all('td', class_="title")
        
    # <td>태그를 리스트에 삽입합니다.
    for td in tds:
        source = td.text
        sources.append(source)

In [62]:
#글의 제목과 게시글 전문을 파싱하는 코드입니다.
driver = webdriver.Chrome('C:\웹 드라이버\chromedriver.exe')
driver.implicitly_wait(1)
driver.get('https://www.jdcenter.com/prcenter/issue/notice.cs')
    
html = driver.page_source
soup = bs4.BeautifulSoup(html, 'html.parser')

sleep(5)    #IDLE TIME은 자신의 인터넷 속도에 맞추어 유동적으로 할당해도 좋습니다.

for i in range(1, 251):
    
    # 2 ~ 25 페이지를 클릭해주는 코드 입니다.
    if i > 10:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(2))
        next_page.click() 
        sleep(4)
    if i > 20:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(3))
        next_page.click() 
        sleep(4)
    if i > 30:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(4))
        next_page.click() 
        sleep(4)
    if i > 40:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/div/a[{}]'.format(5))
        next_page.click() 
        sleep(4)
    if i > 50:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/a[2]')
        next_page.click()
        sleep(4)
    if i > 60:
        next_page = driver.find_element_by_xpath('//*[@id="pagination"]/div/a[3]')
        for k in range(0, (i//10)-5):
            next_page.click()
            sleep(4)
        
        
    #--------------
    
    print(i)
    
    j = i%10
    if j == 0:
        j = 10

    sleep(3)
    post_page = driver.find_element_by_xpath('//*[@id="datatable"]/tbody/tr[{}]/td[2]/a'.format(j))
    post_page.click()

    sleep(3)
    driver.get(driver.current_url)
    html = driver.page_source
    soup = bs4.BeautifulSoup(html, 'html.parser')

    sleep(3)
    h3 = soup.find("h3")
    title.append(h3.text)

    bod = soup.find('div', class_='bod_bd')
    body.append(bod.text)

    driver.back()
    sleep(3)
            

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


AttributeError: 'NoneType' object has no attribute 'text'

In [163]:
#파싱된 텍스트중 \xa0을 지우는 코드 입니다.
for i in range(0, len(body)):
    body[i] = body[i].replace("\xa0","")
    
#리스트에 포함된 택스트가 있다면 택스트를 기준으로 한줄 때어내어 리스트에 삽입하는 코드입니다.
for j in range(2, 50):
    
    cnt += 1
    print(j)
    
    for i in range(0, len(title_names)):
        if title_names[i] in body[j]:
            a = body[j][body[j].index(title_names[i]):]
            a = a[:a.index('\n')]
            title.append(a)
            
    #리스트에 텍스트가 없다면 NONE을 대신 삽입힙니다.
    if len(title) == cnt-1:
        title.append('NONE')

    for i in range(0, len(date_names)):
        if date_names[i] in body[j]:
            a = body[j][body[j].index(date_names[i]):]
            a = a[:a.index('\n')]
            date.append(a)
            break
            
    if len(date) == cnt-1:
        date.append('NONE')

    for i in range(0, len(place_names)):
        if place_names[i] in body[j]:
            a = body[j][body[j].index(place_names[i]):]
            a = a[:a.index('\n')]
            place.append(a)
            
    if len(place) == cnt-1:
        place.append('NONE')

    for i in range(0, len(content_names)):
        if content_names[i] in body[j]:
            a = body[j][body[j].index(content_names[i]):]
            a = a[:a.index('\n')]
            content.append(a)
            
    if len(content) == cnt-1:
        content.append('NONE')

    for i in range(0, len(target_person_name)):
        if target_person_name[i] in body[j]:
            a = body[j][body[j].index(target_person_name[i]):]
            a = a[:a.index('\n')]
            target.append(a)
    if len(target) == cnt-1:
        target.append('NONE')

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


IndexError: list index out of range

In [224]:
dict_list = {'title' : sources,
             'date' : date,
             'place' : place,
             'content' : content,
             'target' : target}

df = pd.DataFrame(dict_list)

#df.to_csv('data_set.csv')

In [39]:
title

["JDC 소셜벤처 지원사업 '낭그늘2기' 지원대상 모집 공고",
 'JDC 사회적경제조직 지원사업 위탁운영기관 공개 모집',
 "JDC 글로벌 인재 맞춤형 일자리 지원사업 'JDC 글로벌 챌린저 3기' 참여자 모집",
 '불용물품(공용차량) 매각(38고 5868)',
 "JDC 이음일자리 플러스 '사회적경제 전문가 과정' 참여자 추가 모집",
 '「JDC 제주관광 플랫폼 사업」관광콘텐츠 공모',
 '한예종과 함께하는 JDC 문화가득 프로그램 참여 요청',
 '2019 디지털트윈 포럼 개최 안내',
 'JDC 마을 공동체 사업 대국민 영상 공모전 개최',
 '제주첨단과학기술단지 입주기업 고용 등 현황 설문조사 협조 요청',
 "JDC 소셜벤처 지원사업 '낭그늘2기' 지원대상 모집 공고",
 '2019 JDC 이음일자리 플러스 참여자 모집',
 '[JDC 드론 시범사업 사업자 공모] 심사결과 알림',
 '2019 JDC 마을공동체사업 공모',
 '2019 제주 ICT 페스티벌 개최 안내(11.09.(토), 제주대학교 체육관)',
 '2019 JDC 복지시설 특장차량 지원 사업 공모',
 'JDC 글로벌 인재 맞춤형 일자리 지원 사업 위탁운영기관 모집 공고',
 'JDC 소셜벤처 지원사업 ‘낭그늘’위탁운영기관 모집 재공고',
 'JDC 드론 시범사업 사업자 공모(재공모) 공고',
 '2019 평화의 섬 제주, JDC Book Fair 개최 안내(10.12(토)), 영어교육도시 배움의 길)',
 'JDC x AI 인공지능 아카데미 참가자 모집',
 '제주첨단과학기술단지 스마트빌딩 임대공고',
 '(~10.11 접수기간 연장) 2019 평화의 섬 제주,  JDC Book Fair 가족 골든벨 퀴즈대회 참가 안내',
 '(~10.11 접수기간 연장)2019 평화의 섬 제주, JDC Book Fair 영어 토론대회 참가 안내',
 '(~10. 11 접수기간 연장)2019 평화의 섬 제주,  JDC Book Fair 영어 스피치 대회 참가 안내',
 '(~10.9 접수기간